In [1]:
import os
import json
import copy
import requests
import pandas as pd
import time

In [11]:
df = pd.read_csv('../Data/leetcode_problems.csv', index_col='question_id')
slugs = list(df['question__title_slug'])
s = requests.Session()
template = {"operationName":"questionData","variables":{"titleSlug":None},"query":"query questionData($titleSlug: String!) {\n  question(titleSlug: $titleSlug) {\n    questionId\n    questionFrontendId\n    boundTopicId\n    title\n    titleSlug\n    content\n    translatedTitle\n    translatedContent\n    isPaidOnly\n    difficulty\n    likes\n    dislikes\n    isLiked\n    similarQuestions\n    contributors {\n      username\n      profileUrl\n      avatarUrl\n      __typename\n    }\n    topicTags {\n      name\n      slug\n      translatedName\n      __typename\n    }\n    companyTagStats\n    codeSnippets {\n      lang\n      langSlug\n      code\n      __typename\n    }\n    stats\n    hints\n    solution {\n      id\n      canSeeDetail\n      paidOnly\n      __typename\n    }\n    status\n    sampleTestCase\n    metaData\n    judgerAvailable\n    judgeType\n    mysqlSchemas\n    enableRunCode\n    enableTestMode\n    enableDebugger\n    envInfo\n    libraryUrl\n    adminUrl\n    __typename\n  }\n}\n"}

In [29]:
def slug_to_qid(slug: str):
    search = df[df['question__title_slug'] == slug]
    assert len(search) <= 1
    return None if len(search) == 0 else int(search.index[0])

In [13]:
def run(slug):
    payload = {"operationName":"questionData","variables":{"titleSlug":slug},"query":"query questionData($titleSlug: String!) {\n  question(titleSlug: $titleSlug) {\n    questionId\n    questionFrontendId\n    boundTopicId\n    title\n    titleSlug\n    content\n    translatedTitle\n    translatedContent\n    isPaidOnly\n    difficulty\n    likes\n    dislikes\n    isLiked\n    similarQuestions\n    contributors {\n      username\n      profileUrl\n      avatarUrl\n      __typename\n    }\n    topicTags {\n      name\n      slug\n      translatedName\n      __typename\n    }\n    companyTagStats\n    codeSnippets {\n      lang\n      langSlug\n      code\n      __typename\n    }\n    stats\n    hints\n    solution {\n      id\n      canSeeDetail\n      paidOnly\n      __typename\n    }\n    status\n    sampleTestCase\n    metaData\n    judgerAvailable\n    judgeType\n    mysqlSchemas\n    enableRunCode\n    enableTestMode\n    enableDebugger\n    envInfo\n    libraryUrl\n    adminUrl\n    __typename\n  }\n}\n"}
    payload_str = json.dumps(payload)
    headers = {
        # "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36",
        "content-length": str(len(payload_str)),
        "content-type": "application/json",
        # "accept-encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Accept": "*/*",
        "origin": "https://leetcode.com",
        "referer": "https://leetcode.com/problems/two-sum/"
    }
    response = s.post("https://leetcode.com/graphql", headers=headers, json=payload)
    question = response.json()['data']['question']
    question_id = int(question['questionId'])
    similar_question_ids = [slug_to_qid(problem['titleSlug']) for problem in json.loads(question['similarQuestions'])]
    topic_tags = [{'name': tag['name'], 'slug': tag['slug']} for tag in question['topicTags']]
    return {
        "question_id": question_id, 
        "topic_tags": topic_tags, 
        "similar_question_ids": similar_question_ids,
        "likes": question["likes"],
        "dislikes": question["dislikes"]
    }

In [39]:
def run_requests():
    results = []
    for index, slug in enumerate(slugs):
        # if index > 5:
        #     break
        # time.sleep(0.2)
        print("{}. {}".format(index, slug))
        results.append(run(slug))
    return results

In [48]:
def run_requests_yield():
    for index, slug in enumerate(slugs):
        # if index > 5:
        #     break
        print("{}. {}".format(index, slug))
        yield run(slug)

In [49]:
def main(yield_=False):
    if yield_:
        for question in run_requests_yield():
            df.loc[question['question_id'], 'likes'] = question['likes']
            df.loc[question['question_id'], 'dislikes'] = question['dislikes']
            df.loc[question['question_id'], 'topic_tags'] = str(question['topic_tags'])
            df.loc[question['question_id'], 'similar_question_ids'] = str(question['similar_question_ids'])    
    else:
        for question in run_requests():
            df.loc[question['question_id'], 'likes'] = question['likes']
            df.loc[question['question_id'], 'dislikes'] = question['dislikes']
            df.loc[question['question_id'], 'topic_tags'] = str(question['topic_tags'])
            df.loc[question['question_id'], 'similar_question_ids'] = str(question['similar_question_ids'])
    df.to_csv("../Data/leetcode_problems.csv")

In [44]:
%time main()

vide-array-into-increasing-sequences
1071. robot-bounded-in-circle
1072. flower-planting-with-no-adjacent
1073. partition-array-for-maximum-sum
1074. longest-duplicate-substring
1075. single-row-keyboard
1076. string-transforms-into-another-string
1077. design-file-system
1078. minimum-cost-to-connect-sticks
1079. last-stone-weight
1080. remove-all-adjacent-duplicates-in-string
1081. longest-string-chain
1082. last-stone-weight-ii
1083. count-substrings-with-only-one-distinct-letter
1084. before-and-after-puzzle
1085. last-substring-in-lexicographical-order
1086. shortest-distance-to-target-color
1087. customers-who-bought-all-products
1088. actors-and-directors-who-cooperated-at-least-three-times
1089. height-checker
1090. grumpy-bookstore-owner
1091. previous-permutation-with-one-swap
1092. distant-barcodes
1093. how-many-apples-can-you-put-into-the-basket
1094. minimum-knight-moves
1095. find-smallest-common-element-in-all-rows
1096. optimize-water-distribution-in-a-village
1097. nu

In [50]:
%time main(yield_=True)

0. two-sum
1. add-two-numbers
2. longest-substring-without-repeating-characters
3. median-of-two-sorted-arrays
4. longest-palindromic-substring
5. zigzag-conversion
6. reverse-integer
7. string-to-integer-atoi
8. palindrome-number
9. regular-expression-matching
10. container-with-most-water
11. integer-to-roman
12. roman-to-integer
13. longest-common-prefix
14. 3sum
15. 3sum-closest
16. letter-combinations-of-a-phone-number
17. 4sum
18. remove-nth-node-from-end-of-list
19. valid-parentheses
20. merge-two-sorted-lists


KeyboardInterrupt: 